In [3]:

#Linear Regression

import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
inputs = np.load("data/classification/inputs.npy")

In [5]:
labels = np.load("data/classification/labels.npy")

In [6]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

In [8]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logisticReg = LogisticRegression(random_state=0).fit(X_train, np.ravel(y_train))
scores = cross_val_score(logisticReg, inputs, np.ravel(labels), cv=5)
scores

array([0.87 , 0.895, 0.86 , 0.915, 0.905])

In [9]:
y_pred = logisticReg.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.91

In [11]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5)).fit(X_train, np.ravel(y_train))
y_pred = svm.predict(X_test)
accuracy_score(y_pred, y_test)

0.905

In [12]:
from sklearn.svm import SVC

svc = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(X_train, np.ravel(y_train))
y_pred = svc.predict(X_test)
accuracy_score(y_pred, y_test)

0.905

In [13]:
from sklearn.svm import NuSVC

svc = make_pipeline(StandardScaler(), NuSVC(nu=0.3)).fit(X_train, np.ravel(y_train))
y_pred = svc.predict(X_test)
accuracy_score(y_pred, y_test)

0.89

In [20]:
from skopt.space import Categorical, Real, Integer
import skopt

search_space = list()
##search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [21]:

from sklearn.model_selection import RepeatedStratifiedKFold
from skopt.utils import use_named_args

# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
	# configure the model with specific hyperparameters
	model = SVC()
	model.set_params(**params)
	# define test harness
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# calculate 5-fold cross validation
	result = cross_val_score(model, inputs, np.ravel(labels), cv=cv, n_jobs=-1, scoring='accuracy')
	# calculate the mean of the scores
	estimate = np.mean(result)
	# convert from a maximizing score to a minimizing score
	return 1.0 - estimate

In [22]:
from skopt import gp_minimize
# perform optimization
result = gp_minimize(evaluate_model, search_space)
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best Accuracy: 0.894
Best Parameters: ['sigmoid', 1, 0.0034323075640702375]
